In [1]:
import pygame
import sys
import time
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class User:
    def __init__(self, full_name, sign):
        self.name = full_name
        self.sign = sign
class Node:
    def __init__(self, state, action):
        self.state = state
        self.children = []
        self.action = action
        self.number_it = []
        self.value = []
        

def count(state):
    state = state.copy()
    width = len(state[0])
    heigth = len(state)
    aux_1=0
    for i in range(len(state[0])):
        for j in range(len(state)): 
            if state[i][j] == 'X' or state[i][j] == 'O':
                aux_1 = aux_1 + 1
    return aux_1    


In [3]:
class TicTacToe:
    def __init__(self):
        start_game = 0
    # Player Function -> Return which player next turn is it (X or O)
    def player(self, board):
        self.board = board.copy()
        count_moves = count(self.board)
        if np.all((board=="")):
            return 'X'
        elif count_moves%2 == 0 or count_moves ==0:
            return 'X'
        else:
            return 'O'
    # Actions function -> should return a set of all of the possible actions that can be taken on a given board (tuple, (i,j))
    def actions(self, board):
        self.board = board.copy()
        self.actions_saved = []
        self.width = len(self.board[0])
        self.heigth = len(self.board)
        for i in range(len(self.board[0])):
            for j in range(len(self.board)):
                if self.board[i][j] != 'X' and self.board[i][j] !='O':
                    self.actions_saved.append((i,j))
        return self.actions_saved  
    # The result function takes a board and an action as input, and should return a new board state, 
    # without modifying the original board
    def result(self, node, action):
        self.matrix = node.copy()
        self.action = action
        self.play_next = self.player(self.matrix)
        self.row, self.col = int(self.action[0]), int(self.action[1])
        self.matrix[self.row][self.col] = self.play_next
        return self.matrix
    
    #The winner function should accept a board as input, and return the winner of the board if there is one
    def winner(self, board):
        self.board = board.copy()
        signs = ['O', 'X']
        for sign in signs:
            if self.board[0,0] == sign and self.board[1,1] ==sign and self.board[2,2] ==sign:
                return sign
            elif self.board[2,0] == sign and self.board[1,1] ==sign and self.board[0,2] ==sign:
                return sign
            for j in range(len(self.board)):
                if self.board[j,0] == sign and self.board[j,1] ==sign and self.board[j,2] == sign:
                    return sign
            for i in range(len(self.board)):
                if self.board[0,i] == sign and self.board[1,i] == sign and self.board[2,i] == sign:
                    return sign
        return None
    
    # The terminal function should accept a board as input, and return a boolean value indicating whether the game is over.
    def terminal(self, board):
        self.board = board.copy()
        self.winner_sign = self.winner(self.board)
        if self.winner_sign == 'X' or self.winner_sign == 'O':
            return True
        elif np.any(self.board==''):
            return False
        else:
            return True
        
    # The utility function should accept a terminal board as input and output the utility of the board.
    def utility(self, board):
        self.board = board.copy()
        if self.terminal(self.board):
            self.winner_sign = self.winner(self.board)
            if not self.winner_sign:
                self.utility_value = 0
            elif self.winner_sign == 'X':
                self.utility_value = 1
            else:
                self.utility_value = -1
        else:
            return None
        return self.utility_value
        
    def create_children(self, root_node, sign):        
        possible_actions = self.actions(root_node.state)
        count_state = count(root_node.state) + 1
        self.sign = sign
        # First we get the first line of child of the board state
        for action in possible_actions:
            root_node.children.append(Node(state = self.result(root_node.state, action), action = action))
        # Recursion that goes throguh all the child and options from the root_state.
        #for i in range(len(root_node.children)):
        if self.sign == 'O':
            value_o = float('inf')
            value_x = float('-inf')
            for i in range(len(root_node.children)):
                if self.terminal(root_node.children[i].state):
                    state_value = self.utility(root_node.children[i].state)
                    root_node.children[i].value.append(state_value)
                else:
                    root_node.children[i] = self.create_children(root_node.children[i], self.sign)

                #Transform "list" with the utility value to a string and then to a float

                children = str(root_node.children[i].value)
                children = children.replace('[', '')
                children = children.replace(']', '')

                if count_state%2==0 and float(children) < value_o:
                    value_o = float(children)
                    root_node.value = root_node.children[i].value
                elif float(children) > value_x and count_state%2!=0:
                    value_x = float(children)
                    root_node.value = root_node.children[i].value
        elif self.sign == 'X':
            value_o = float('inf')
            value_x = float('-inf')
            
            for i in range(len(root_node.children)):
                if self.terminal(root_node.children[i].state):
                    state_value = self.utility(root_node.children[i].state)
                    root_node.children[i].value.append(state_value)
                else:
                    root_node.children[i] = self.create_children(root_node.children[i], self.sign)

                #Transform "list" with the utility value to a string and then to a float

                children = str(root_node.children[i].value)
                children = children.replace('[', '')
                children = children.replace(']', '')
                
                if (count_state%2!=0 or count_state ==0) and float(children) > value_x:
                    value_x = float(children)
                    root_node.value = root_node.children[i].value
                elif float(children) < value_o and count_state%2==0:
                    value_o = float(children)
                    root_node.value = root_node.children[i].value
            
        self.node = root_node
        return self.node
    def evaluate_children(self, root_node, sign):
        self.node_nextplay = root_node
        count_state = count(root_node.state) + 1
        if sign=='O':
            value = float('inf')
        elif sign == 'X':
            value = float('-inf')
        for i in range(len(root_node.children)):
            children = str(root_node.children[i].value)
            children = children.replace('[', '')
            children = children.replace(']', '')
            if float(children) < value and sign == 'O':
                value = float(children)
                self.node_nextplay = root_node.children[i]
            elif float(children) > value and sign == 'X':
                value = float(children)
                self.node_nextplay = root_node.children[i]
        return self.node_nextplay
    
    # The minimax function should take a board as input, and return the optimal move for the player to move on that board.
    def minimax(self, board):
        self.board = board.copy()  
        if self.terminal(self.board):
            return None
        # If next player is O, then it need to min utility value
        if self.player(self.board) == 'O':
            root_node = Node(state = self.board, action = None)
            root_node = self.create_children(root_node, 'O')
            self.node_nextplay = self.evaluate_children(root_node, 'O')
            return self.node_nextplay
        elif self.player(self.board) == 'X':
            root_node = Node(state = self.board, action = None)
            root_node = self.create_children(root_node, 'X')
            self.node_nextplay = self.evaluate_children(root_node, 'X')
            return self.node_nextplay   
                    



In [4]:
pygame.init()
size = width, height = 600, 400

# Colors
black = (0, 0, 0)
white = (255, 255, 255)

screen = pygame.display.set_mode(size)

mediumFont = pygame.font.SysFont("OpenSans-Regular.ttf", 28)
largeFont = pygame.font.SysFont("OpenSans-Regular.ttf", 40)
moveFont = pygame.font.SysFont("OpenSans-Regular.ttf", 60)


TicTacToe_game = TicTacToe()
#user1 = User(input("Full Name: "), input("Sign (X or O): "))
#
#TicTacToe_game.game_start(user1,board)

user = None
board = np.array([["","",""],["","",""],["","",""]], dtype=str)
ai_turn = False

while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

    screen.fill(black)

    # Let user choose a player.
    if user is None:

        # Draw title
        title = largeFont.render("Play Tic-Tac-Toe", True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 50)
        screen.blit(title, titleRect)

        # Draw buttons
        playXButton = pygame.Rect((width / 8), (height / 2), width / 4, 50)
        playX = mediumFont.render("Play as X", True, black)
        playXRect = playX.get_rect()
        playXRect.center = playXButton.center
        pygame.draw.rect(screen, white, playXButton)
        screen.blit(playX, playXRect)

        playOButton = pygame.Rect(5 * (width / 8), (height / 2), width / 4, 50)
        playO = mediumFont.render("Play as O", True, black)
        playORect = playO.get_rect()
        playORect.center = playOButton.center
        pygame.draw.rect(screen, white, playOButton)
        screen.blit(playO, playORect)

        # Check if button is clicked
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1:
            mouse = pygame.mouse.get_pos()
            if playXButton.collidepoint(mouse):
                time.sleep(0.2)
                user = 'X'
                user1 = User(input("Full Name: "), user)
            elif playOButton.collidepoint(mouse):
                time.sleep(0.2)
                user ='O'
                user1 = User(input("Full Name: "), user)

    else:

        # Draw game board
        tile_size = 80
        tile_origin = (width / 2 - (1.5 * tile_size),
                       height / 2 - (1.5 * tile_size))
        tiles = []
        for i in range(3):
            row = []
            for j in range(3):
                rect = pygame.Rect(
                    tile_origin[0] + j * tile_size,
                    tile_origin[1] + i * tile_size,
                    tile_size, tile_size
                )
                pygame.draw.rect(screen, white, rect, 3)

                if board[i][j] != '':
                    move = moveFont.render(board[i][j], True, white)
                    moveRect = move.get_rect()
                    moveRect.center = rect.center
                    screen.blit(move, moveRect)
                row.append(rect)
            tiles.append(row)

        game_over = TicTacToe_game.terminal(board)
        player = TicTacToe_game.player(board)

        # Show title
        if game_over:
            winner = TicTacToe_game.winner(board)
            if winner is None:
                title = f"Game Over: Tie."
            elif user1.sign == winner:
                title = f"Game Over: {user.name} wins."
            else:
                title = f"Game Over: Computer wins."
        elif user == player:
            title = f"Play as {user}"
        else:
            title = f"Computer thinking..."
        title = largeFont.render(title, True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 30)
        screen.blit(title, titleRect)

        # Check for AI move
        if user1.sign != player and not game_over:
            if ai_turn:
                time.sleep(0.5)
                move = TicTacToe_game.minimax(board)
                board = TicTacToe_game.result(board, move.action)
                ai_turn = False
            else:
                ai_turn = True

        # Check for a user move
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1 and user == player and not game_over:
            mouse = pygame.mouse.get_pos()
            for i in range(3):
                for j in range(3):
                    if (board[i][j] == '' and tiles[i][j].collidepoint(mouse)):
                        board = TicTacToe_game.result(board, (i, j))

        if game_over:
            againButton = pygame.Rect(width / 3, height - 65, width / 3, 50)
            again = mediumFont.render("Play Again", True, black)
            againRect = again.get_rect()
            againRect.center = againButton.center
            pygame.draw.rect(screen, white, againButton)
            screen.blit(again, againRect)
            click, _, _ = pygame.mouse.get_pressed()
            if click == 1:
                mouse = pygame.mouse.get_pos()
                if againButton.collidepoint(mouse):
                    time.sleep(0.2)
                    user = None
                    board = np.array([["","",""],["","",""],["","",""]], dtype=str)
                    ai_turn = False

    pygame.display.flip()

Full Name: Carlos
Full Name: Carlos


SystemExit: 

C:\Users\fadom\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
